# Sheep

In [ ]:
# environment init
!pip install ../input/panda-essential/pretrained-models.pytorch-master/pretrained-models.pytorch-master/
!pip install ../input/landmark-additional-packages/EfficientNet-PyTorch/EfficientNet-PyTorch-master/
!pip install ../input/../input/landmark-additional-packages/timm-0.3.4-py3-none-any.whl

!mkdir /root/.cache/torch
!mkdir /root/.cache/torch/hub/
!mkdir /root/.cache/torch/hub/checkpoints
!cp -r ../input/landmark-additional-packages/rwightman_gen-efficientnet-pytorch_master/rwightman_gen-efficientnet-pytorch_master /root/.cache/torch/hub/
!cp -r ../input/landmark-additional-packages/tf_efficientnet_* /root/.cache/torch/hub/checkpoints
!cp -r ../input/landmark-additional-packages/se* /root/.cache/torch/hub/checkpoints
!cp -r ../input/landmark-additional-packages/resnet200d* /root/.cache/torch/hub/checkpoints
!cp -r ../input/landmark-additional-packages/dense* /root/.cache/torch/hub/checkpoints

In [ ]:
!ls /root/.cache/torch/hub/checkpoints

In [ ]:
!mkdir leaf
!cp -r ../input/leaf-search-7fa0ec2/leaf-search/* leaf
!cp ../input/leaf-search-7fa0ec2/leaf-search/notebook_infer.py .

In [ ]:
import glob
from subprocess import call
import numpy as np
import pandas as pd
get_score = lambda x: float(x.split('/')[-1][-10:-4])

In [ ]:
cmd = ('python notebook_infer.py --model={} ' + 
        '--config={} ' +
       '--df=../input/cassava-leaf-disease-classification/sample_submission.csv --output=test')

## Model list
```python
'../input/leaf-0210-four-models/0129_dense_dpp_cutmix_744_n1_sin_aug_cut_1_fn/0129_dense_dpp_cutmix_744_n1_sin_aug_cut_1_fn'
'../input/leaf-0129-eff-se50-three-model/0129_eff_dpp_cutmix_eff_16_ext_v1_f0_n1_sin_aug_cut_1_744_beta6_f/0129_eff_dpp_cutmix_eff_16_ext_v1_f0_n1_sin_aug_cut_1_744_beta6_f1.yaml'
'../input/leaf-0210-four-models/0209_upload_eb7_eb7_fn/0209_upload_eb7_eb7_f0.yaml'
'../input/leaf-0210-four-models/0209_upload_r200d_light_beta6_800_16_r200d_800_16_beta6_fn_16/0209_upload_r200d_light_beta6_800_16_r200d_800_16_beta6_fn_16'
'../input/leaf-0210-four-models/0209_upload_se50_full_se50_744_sin_full_fn/0209_upload_se50_full_se50_744_sin_full_fn'
'../input/leaf-0129-eff-se50-three-model/upload_clean_ext_no_ext_sin_clean_600_drop_snapmix_32_ext_v1_fn_n1_sin_aug_cut_0.25.yaml/upload_clean_ext_no_ext_sin_clean_600_drop_snapmix_32_ext_v1_fn_n1_sin_aug_cut_0.25.yaml'

```

In [ ]:
model_1 = '../input/leaf-0210-four-models/0129_dense_dpp_cutmix_744_n1_sin_aug_cut_1_fn/0129_dense_dpp_cutmix_744_n1_sin_aug_cut_1_fn'
model_2 = '../input/leaf-0129-eff-se50-three-model/0129_eff_dpp_cutmix_eff_16_ext_v1_f0_n1_sin_aug_cut_1_744_beta6_f1/0129_eff_dpp_cutmix_eff_16_ext_v1_f0_n1_sin_aug_cut_1_744_beta6_f1.yaml'
model_3 = '../input/leaf-0210-four-models/0209_upload_eb7_eb7_fn/0209_upload_eb7_eb7_f0.yaml'
model_4 = '../input/leaf-0210-four-models/0209_upload_r200d_light_beta6_800_16_r200d_800_16_beta6_fn_16/0209_upload_r200d_light_beta6_800_16_r200d_800_16_beta6_fn_16'
model_5 = '../input/leaf-0210-four-models/0209_upload_se50_full_se50_744_sin_full_fn/0209_upload_se50_full_se50_744_sin_full_fn'
model_6 = '../input/leaf-0129-eff-se50-three-model/upload_clean_ext_no_ext_sin_clean_600_drop_snapmix_32_ext_v1_fn_n1_sin_aug_cut_0.25.yaml/upload_clean_ext_no_ext_sin_clean_600_drop_snapmix_32_ext_v1_fn_n1_sin_aug_cut_0.25.yaml'

## Preds

In [ ]:
sheep_models = []
for m in [model_1, model_2, model_3, model_4, model_5, model_6]:
    preds = []
    for f in range(5):
        mdl = glob.glob(f'{m}/f{f}/*.pth')[0]
        cfg = glob.glob(f'{m}/f{f}/*.json')[0]
        call(cmd.format(mdl, cfg), shell=True)
        preds.append(np.load('test.npy'))
    pred = np.zeros_like(preds[0])
    for x in preds:
        pred += x
    pred = pred / 5
    sheep_models.append(pred)

## Combine Preds

In [ ]:
sheep_models

# sin

In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import pandas as pd
import numpy as np
import time
import cv2
import PIL.Image
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import albumentations
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import joblib
import lightgbm as lgb
import timm
import glob
device = torch.device('cuda') 

In [ ]:
class CLDDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        image = cv2.imread(row.filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        
        image = image.astype(np.float32)
        image = image.transpose(2,0,1)
        if self.mode == 'test':
            return torch.tensor(image).float()
        else:
            return torch.tensor(image).float(), torch.tensor(row.label).float()

In [ ]:


class CustomTimmModel(nn.Module):
    def __init__(self, backbone, out_dim, pretrained=True):
        super().__init__()
        self.model = timm.create_model(backbone, pretrained=pretrained)
        if 'efficientnet' in backbone or 'densenet' in backbone:
            in_ch = self.model.classifier.in_features
            self.model.classifier = nn.Identity() 
        elif 'resnext' in backbone or 'resnet' in backbone or 'resnest' in backbone:
            in_ch = self.model.fc.in_features
            self.model.fc = nn.Identity() 
        elif 'vit' in backbone:
            in_ch = self.model.head.in_features
            self.model.head = nn.Identity()
        elif 'csp' in backbone:
            in_ch = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
            
        self.myfc = nn.Linear(in_ch, out_dim)
    def forward(self, x):
        x = self.model(x)
        x = self.myfc(x)
        return x



class CLDResNext(nn.Module):

    def __init__(self, backbone, out_dim, pretrained=True):
        super().__init__()
        self.model = timm.create_model(backbone, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, out_dim)

    def forward(self, x):
        x = self.model(x)
        return x

class CLDVit(nn.Module):
    def __init__(self, backbone, out_dim, pretrained=False):
        super().__init__()
        self.model = timm.create_model(backbone, pretrained=pretrained)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, out_dim)

    def forward(self, x):
        x = self.model(x)
        return x



def inference_func(test_loader):
    model.eval()
    print('Predicting...')
    LOGITS = []
    PREDS = []
    
    with torch.no_grad():
        for images in test_loader:
            x = images.to(device)
            logits = model(x)
            LOGITS.append(logits.cpu())
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()
        LOGITS = torch.cat(LOGITS).cpu().numpy()
    return PREDS



test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/test_images', f'{x}'))



In [ ]:
sin_models = []

## Vit

In [ ]:
batch_size = 32
image_size = 384
model_type = ['vit_base_patch16_384'] * 5
model_path = ['../input/cass-vit-base-384/fold0.pth', 
              '../input/cass-vit-base-384/fold1.pth', 
              '../input/cass-vit-base-384/fold2.pth',
              '../input/cass-vit-base-384/fold3.pth',
              '../input/cass-vit-base-384/fold4.pth']

transforms_valid = albumentations.Compose([
    albumentations.CenterCrop(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CLDVit(model_type[i], out_dim=5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]

In [ ]:
sin_models.append(test_preds)

## B4ns_512

In [ ]:
batch_size = 32
image_size = 512
model_type = ['tf_efficientnet_b4_ns'] * 5
model_path = ['../input/b4ns-512/fold0.pth', 
              '../input/b4ns-512/fold1.pth', 
              '../input/b4ns-512/fold2.pth',
              '../input/b4ns-512/fold3.pth',
              '../input/b4ns-512/fold4.pth']

transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CustomTimmModel(model_type[i], out_dim=5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]


In [ ]:
sin_models.append(test_preds)

## Resnest50

In [ ]:
batch_size = 24
image_size = 700
model_type = ['resnest50d_1s4x24d'] * 5
model_path = ['../input/resnest50-pl-cv898/fold0.pth', 
              '../input/resnest50-pl-cv898/fold1.pth', 
              '../input/resnest50-pl-cv898/fold2.pth',
              '../input/resnest50-pl-cv898/fold3.pth',
              '../input/resnest50-pl-cv898/fold4.pth',]

transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CustomTimmModel(model_type[i], out_dim=5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]

In [ ]:
sin_models.append(test_preds)

## Darknet

In [ ]:
batch_size = 32
image_size = 648
model_type = ['cspdarknet53'] * 5
model_path = ['../input/cspdarknet53/fold0.pth', 
              '../input/cspdarknet53/fold1.pth', 
              '../input/cspdarknet53/fold2.pth',
              '../input/cspdarknet53/fold3.pth',
              '../input/cspdarknet53/fold4.pth',]

transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CustomTimmModel(model_type[i], out_dim=5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]

In [ ]:
sin_models.append(test_preds)

In [ ]:
sin_models = [np.mean(i, axis=0) for i in sin_models]

# Holy Chen

In [ ]:
class CassavaClassifier(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        ### EffNet
        if model_arch == 'tf_efficientnet_b4_ns' or model_arch == 'tf_efficientnet_b5_ns':
            num_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(num_features, num_classes)
           
        ### Vit
        if model_arch == 'vit_base_patch16_384':
            num_features = self.model.head.in_features
            self.model.head = nn.Linear(num_features, num_classes)

        ### ResNet
        if model_arch == 'resnext50d_32x4d' or model_arch == 'resnet200d_320' or model_arch == 'seresnet152d_320':
            num_features = self.model.fc.in_features
            self.model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
chen_models = []

## Vit

In [ ]:
batch_size = 32
image_size = 384
model_type = ['vit_base_patch16_384'] * 5
model_path = ['../input/cldcvitb16sin/models/vit_base_patch16_384_fold0_best.ckpt', 
              '../input/cldcvitb16sin/models/vit_base_patch16_384_fold1_best.ckpt', 
              '../input/cldcvitb16sin/models/vit_base_patch16_384_fold2_best.ckpt',
              '../input/cldcvitb16sin/models/vit_base_patch16_384_fold3_best.ckpt',
              '../input/cldcvitb16sin/models/vit_base_patch16_384_fold4_best.ckpt',]

transforms_valid = albumentations.Compose([
    albumentations.CenterCrop(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CassavaClassifier(model_type[i], 5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]

In [ ]:
chen_models.append(test_preds)

## resnet200d-320

In [ ]:
batch_size = 32
image_size = 512
model_type = ['resnet200d_320'] * 5
model_path = ['../input/cldcresnet200d-320sin/models/resnet200d_320_fold0_best.ckpt', 
              '../input/cldcresnet200d-320sin/models/resnet200d_320_fold1_best.ckpt', 
              '../input/cldcresnet200d-320sin/models/resnet200d_320_fold2_best.ckpt',
              '../input/cldcresnet200d-320sin/models/resnet200d_320_fold3_best.ckpt',
              '../input/cldcresnet200d-320sin/models/resnet200d_320_fold4_best.ckpt',]

transforms_valid = albumentations.Compose([
    albumentations.CenterCrop(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CassavaClassifier(model_type[i], 5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]

In [ ]:
chen_models.append(test_preds)

## tf_efficientnet_b4_ns

In [ ]:
batch_size = 32
image_size = 512
model_type = ['tf_efficientnet_b4_ns'] * 5
model_path = ['../input/cldcb4sin/models/tf_efficientnet_b4_ns_fold0_best.ckpt', 
              '../input/cldcb4sin/models/tf_efficientnet_b4_ns_fold1_best.ckpt', 
              '../input/cldcb4sin/models/tf_efficientnet_b4_ns_fold2_best.ckpt',
              '../input/cldcb4sin/models/tf_efficientnet_b4_ns_fold3_best.ckpt',
              '../input/cldcb4sin/models/tf_efficientnet_b4_ns_fold4_best.ckpt',]

transforms_valid = albumentations.Compose([
    albumentations.CenterCrop(image_size, image_size),
    albumentations.Normalize()
])


test_dataset = CLDDataset(test, 'test', transform=transforms_valid)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=4)

test_preds = []
for i in range(len(model_type)):
    model = CassavaClassifier(model_type[i], 5, pretrained=False)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
    test_preds += [inference_func(test_loader)]

In [ ]:
chen_models.append(test_preds)

In [ ]:
chen_models = [np.mean(i, axis=0) for i in chen_models]

## Stacking

In [ ]:
sin_models.extend(sheep_models)

In [ ]:
sin_models.extend(chen_models)

In [ ]:
preds_overall = np.hstack(np.array(sin_models).squeeze())

In [ ]:
preds = []
for model in glob.glob('../input/13model-stacking/*.pkl'):
    gbm = joblib.load(model)
    try:
        preds += [gbm.predict(preds_overall)]
    except:
        preds += [gbm.predict(preds_overall.reshape(1,-1))]

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.label = np.argmax(np.mean(preds, axis=0), axis=1)
submission.to_csv('submission.csv', index=False)